In [112]:
from typing import List
from copy import deepcopy
import numpy as np

# Parameters

In [113]:
global u, d, r, d_r, p_tilde, q_tilde, T, S0, K, X0
global stock_price_tree, option_value_tree, hedge_tree, portfolio_value_tree
u = 2 
d = 0.5 
r = 0.25 # interest rate
d_r = 1/(1+r) # discount rate
p_tilde = (1+r-d) / (u-d) 
q_tilde = (u-(1+r)) / (u-d)
T = 2 # expiration date 
S0 = 4 # initial stock price
K = 5  # strike price

"""
    option_type denotes the type of option
    1: European call option
    2: American call option
    -1: European put option
    -2: American put option
"""
option_type = -2

In [114]:
def stock_price_tree_generator() -> List[List[float]]:
    # tree[i][j] is value in T i w.r.t j units of Tail(down)
    stock_price_tree = [[0 for j in range(i+1)] for i in range(T+1)]
    stock_price_tree[0][0] = S0
    for i in range(1, T+1):
        for j in range(i):
            stock_price_tree[i][j] = stock_price_tree[i-1][j] * u
        stock_price_tree[i][-1] = stock_price_tree[i-1][-1] * d
    return stock_price_tree

def option_value_tree_generator() -> List[List[float]]:
    option_value_tree = deepcopy(stock_price_tree)
    # generate a European option tree
    if abs(option_type == 1):
        for j in range(T+1):
            option_value_tree[-1][j] = max(option_type * (stock_price_tree[-1][j] - K), 0)
        for i in range(T-1,-1,-1):
            for j in range(i+1):
                option_value_tree[i][j] = d_r * (p_tilde*option_value_tree[i+1][j] + q_tilde*option_value_tree[i+1][j+1])
    # generate a American option tree
    else:
        for j in range(T+1):
            option_value_tree[-1][j] = max(option_type/2 * (stock_price_tree[-1][j] - K), 0)
        for i in range(T-1, -1, -1):
            for j in range(i+1):
                cur_payoff = max(option_type/2 * (stock_price_tree[i][j] - K), 0)
                option_value_tree[i][j] = max(cur_payoff,\
                                              d_r * (p_tilde*option_value_tree[i+1][j] + q_tilde*option_value_tree[i+1][j+1]))            
    return option_value_tree

def hedge_tree_generator() -> List[List[float]]:
    hedge_tree = [[0 for j in range(i+1)] for i in range(T)]
    for i in range(T):
        for j in range(i+1):
            delta_S = stock_price_tree[i+1][j] - stock_price_tree[i+1][j+1]
            delta_V = option_value_tree[i+1][j] - option_value_tree[i+1][j+1]
            hedge_tree[i][j] = delta_V / delta_S
    return hedge_tree

def portfolio_value_tree_generator() -> List[List[float]]:
    portfolio_value_tree = [[0 for j in range(i+1)] for i in range(T+1)]
    portfolio_value_tree[0][0] = X0
    for i in range(1, T+1):
        for j in range(i):
            portfolio_value_tree[i][j] = hedge_tree[i-1][j]*stock_price_tree[i][j] +\
                  (1+r)*(portfolio_value_tree[i-1][j] - hedge_tree[i-1][j]*stock_price_tree[i-1][j])
        portfolio_value_tree[i][-1] = hedge_tree[i-1][-1]*stock_price_tree[i][-1] +\
            (1+r)*(portfolio_value_tree[i-1][-1] - hedge_tree[i-1][-1] * stock_price_tree[i-1][-1])
    return portfolio_value_tree


In [115]:
stock_price_tree = stock_price_tree_generator()
option_value_tree = option_value_tree_generator()
hedge_tree = hedge_tree_generator()
# X0 is the initial money
X0 = option_value_tree[0][0]
portfolio_value_tree = portfolio_value_tree_generator()

In [116]:
print("stock price: ", stock_price_tree)
print("option value: ", option_value_tree)
print("hedge: ", hedge_tree)
print("portfolio: ", portfolio_value_tree)

stock price:  [[4], [8, 2.0], [16, 4.0, 1.0]]
option value:  [[1.36], [0.4, 3.0], [0, 1.0, 4.0]]
hedge:  [[-0.43333333333333335], [-0.08333333333333333, -1.0]]
portfolio:  [[1.36], [0.40000000000000036, 3.0000000000000004], [2.220446049250313e-16, 2.25, 5.25]]
